In [ ]:
# Notebook 6: Detección y reconocimiento de caras EN VIDEOS

# Importamos las librerias necesarias
import cv2 # Importamos OpenCV para procesamiento de imágenes
import numpy as np # Importamos numpy para manejo de arrays
import matplotlib.pyplot as plt # Importamos matplotlib para visualización
from google.colab.patches import cv2_imshow # Importamos cv2_imshow para mostrar imágenes en Colab
# Importar la camara de mi computadora
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import google

In [ ]:
# 1. Descargar los clasificadores en cascada pre entrenados para detección facial
# Estos son archivos XML con patrones pre entrenados para detección
!wget -O haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!wget -O haarcascade_eye.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml

--2025-10-06 22:27:49--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.05s   

2025-10-06 22:27:49 (18.9 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]

--2025-10-06 22:27:49--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... c

In [ ]:
# Cargar los clasificadores cascada
# Cargamos el clasificador para rostros frontales
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Cargamso el clasificador para ojos
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [ ]:
# 2. Cargar una imagen de ejemplo con rostros
from urllib.request import urlopen # Para abir URLs

# URL de un video con muchos rostros pasando
# url = 'https://youtu.be/NKrmicPqy2M?si=cPbsRjNIFlV0FOYq'

# Leemos el video

# Option 1: Read from a video file
# cap = cv2.VideoCapture('your_video.mp4') # Replace 'your_video.mp4' with the path to your video file

# Option 2: Read from the live camera (usually index 0)
cap = cv2.VideoCapture(0)

# Capture a single frame before the loop starts
ret, frame = cap.read()

if not ret:
    print("Error: Could not read frame from video source.")
else:
    while True:
        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Draw rectangles around the detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

        # Display the frame with detected faces
        cv2_imshow(frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

# Display the captured frame with detections after the loop finishes
if ret: # Only display if a frame was successfully captured
    print("Displaying the captured frame with detections:")
    cv2_imshow(frame)

Error: Could not read frame from video source.


In [ ]:
# Helper function to stream video from the webcam
def video_stream():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var captureContext;
    var displayCanvas;
    var displayContext;

    var pendingResolve = null;
    var shutdown = false;

    function startup() {
      displayCanvas = document.createElement('canvas');
      displayContext = displayCanvas.getContext('2d');

      captureCanvas = document.createElement('canvas');
      captureContext = captureCanvas.getContext('2d');

      video = document.createElement('video');
      video.setAttribute('playsinline', '');
      video.autoplay = true;
      video.muted = true;

      div = document.createElement('div');
      div.appendChild(video);
      document.body.appendChild(div);

      stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;

      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Start capturing and displaying frames
      requestAnimationFrame(processFrame);
    }

    function processFrame() {
      if (shutdown) {
        stream.getVideoTracks()[0].stop();
        video.remove();
        div.remove();
        return;
      }

      captureContext.drawImage(video, 0, 0, video.videoWidth, video.videoHeight);
      var data = captureCanvas.toDataURL('image/jpeg', 0.8);

      displayContext.drawImage(video, 0, 0, displayCanvas.width, displayCanvas.height);
      // You can add processing here if needed before displaying

      google.colab.kernel.invokeWithCallback({
        'functionName': 'handle_frame',
        'args': [data],
        'autoAck': true
      });

      requestAnimationFrame(processFrame);
    }

    function stop() {
      shutdown = true;
    }

    startup();
    ''')
    display(js)
    print('Streaming started. Press stop_video() to stop.')

def stop_video():
  # Access the google object via the global scope in Javascript
  google.colab.output.eval_js('google.colab.output.eval_js("stop()")')
  print('Streaming stopped.')

In [ ]:
# This function will be called by the Javascript code for each frame
def handle_frame(data):
    # Decode the image using OpenCV
    binary = b64decode(data.split(',')[1])
    nparr = np.frombuffer(binary, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if frame is None:
        print("Error: Could not decode frame.")
        return

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

    # Display the frame with detected faces
    cv2_imshow(frame)

In [ ]:
# Start the video stream
video_stream()

<IPython.core.display.Javascript object>

Streaming started. Press stop_video() to stop.


Run the cell above to start the real-time video stream with face and eye detection.
To stop the stream, run the cell below.

In [ ]:
# Stop the video stream
stop_video()

MessageError: TypeError: google.colab.output.eval_js is not a function